# Marginal Social Costs for Health-Related Emissions

In [ ]:
import os
import pandas as pd

# import from cmu-tare-model package
from config import PROJECT_ROOT
from cmu_tare_model.utils.inflation_adjustment import cpi_ratio_2023_2006

def process_rcm_data(filename, PROJECT_ROOT):
    """
    Processes the RCM data for use in creating a health-related emissions marginal social cost lookup dictionary.
    """
    # Step 1: Load CSV
    relative_path = os.path.join("cmu_tare_model", "data", "marginal_social_costs", filename)
    file_path = os.path.join(PROJECT_ROOT, relative_path)
    df_rcm_msc_data = pd.read_csv(file_path)

    # Step 2: Filter rows where the season column is 'annual'
    df_rcm_msc_data = df_rcm_msc_data[df_rcm_msc_data['season'] == 'annual']

    # Step 3: Convert 'fips' to string and pad with leading zeros (to ensure a 5-digit format)
    df_rcm_msc_data['fips'] = df_rcm_msc_data['fips'].astype(str).str.zfill(5)

    # Step 4: create a new column 'state' from the 'state_abbr' column
    df_rcm_msc_data['state'] = df_rcm_msc_data['state_abbr']

    # Step 5: Inflate from USD2006 to USD2023
    # CACES Default VSL is 7.4M USD2006
    # We took the New EPA VSL 11.3 and converted it to USD2006 using the CPI ratio
    # The custom VSL is input in USD2006
    # We then inflated the output to USD2023 using the CPI ratio
    df_rcm_msc_data['damage_usd2023'] = df_rcm_msc_data['damage'] * cpi_ratio_2023_2006

    # Step 6: Return df_rcm_msc_data so that it can be processed into a nested lookup dictionary
    return df_rcm_msc_data

def create_lookup_nested(df_rcm_msc_data):
    """
    Creates a nested lookup dictionary with keys as (fips, state),
    then model, then pollutant mapping to damage.
    
    Assumptions:
      - 'fips' is a padded string,
      - 'state_abbr' is provided, --> 'state' is created from 'state_abbr',
      - 'model' identifies the damage model,
      - 'pollutant' is already in uppercase.

    ======================================================================================================================
    LOOKUP DICTIONARY FORMAT
    ======================================================================================================================
    lookup = {
        (fips, state): {
            model: {
                pollutant: damage,
                ...
            },
            ...
        },
        ...
    }
    ======================================================================================================================
    """
    lookup_health_rcm_msc = {}
    for _, row in df_rcm_msc_data.iterrows():
        county_key = (row['fips'], row['state'])
        model = row['model']
        pollutant = row['pollutant']
        damage = row['damage_usd2023']
        
        # Initialize dictionary for the county if needed.
        if county_key not in lookup_health_rcm_msc:
            lookup_health_rcm_msc[county_key] = {}
        # Initialize dictionary for the model if needed.
        if model not in lookup_health_rcm_msc[county_key]:
            lookup_health_rcm_msc[county_key][model] = {}
        # Set the damage value for the pollutant.
        lookup_health_rcm_msc[county_key][model][pollutant] = damage
        
    return lookup_health_rcm_msc

Project root directory: c:\users\14128\research\cmu-tare-model
Retrieved data for filename: bls_cpiu_2005-2023.xlsx
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\inflation_data\bls_cpiu_2005-2023.xlsx
    year  cpiu_annual
0   2005      195.300
1   2006      201.600
2   2007      207.342
3   2008      215.303
4   2009      214.537
5   2010      218.056
6   2011      224.939
7   2012      229.594
8   2013      232.957
9   2014      236.736
10  2015      237.017
11  2016      240.007
12  2017      245.120
13  2018      251.107
14  2019      255.657
15  2020      258.811
16  2021      270.970
17  2022      292.655
18  2023      304.702


## Fossil Fuels are Ground-Level Emissions

In [2]:
# ======================================================================================================================
# FOSSIL FUEL COMBUSTION:
# Load the data for Fossil Fuel Marginal Social Costs 
# Emissions from Fossil Fuels are considered Ground-Level
# ======================================================================================================================

# Fossil Fuels
# Ground Level, ACS C-R Function
df_ground_acs = process_rcm_data("rcm_msc_county_vsl857_usd2006_ground_acs.csv", PROJECT_ROOT)

# Ground Level, H6C C-R Function
df_ground_h6c = process_rcm_data("rcm_msc_county_vsl857_usd2006_ground_h6c.csv", PROJECT_ROOT)

print(f"""
======================================================================================================================
FOSSIL FUEL COMBUSTION:
Load the data for Fossil Fuel Marginal Social Costs 
Emissions from Fossil Fuels are considered Ground-Level
======================================================================================================================
Model: RCM
Output Area: FIPS
Geography: Counties
Pollutant: NOX,SO2,PM25
VSL: 8.57M (Inflated from 11.3M USD2021 to 8.57M USD2006)
Stack Height: ground level

C-R Function: ACS or H6C
======================================================================================================================
- Clean the data so that only annual values are retained. 
- Convert 'fips' to string and pad with leading zeros (to ensure a 5-digit format)

DATAFRAME: Ground Level, ACS C-R Function

{df_ground_acs}
      
DATAFRAME: Ground Level, H6C C-R Function

{df_ground_h6c}
      
""")


FOSSIL FUEL COMBUSTION:
Load the data for Fossil Fuel Marginal Social Costs 
Emissions from Fossil Fuels are considered Ground-Level
Model: RCM
Output Area: FIPS
Geography: Counties
Pollutant: NOX,SO2,PM25
VSL: 8.57M (Inflated from 11.3M USD2021 to 8.57M USD2006)
Stack Height: ground level

C-R Function: ACS or H6C
- Clean the data so that only annual values are retained. 
- Convert 'fips' to string and pad with leading zeros (to ensure a 5-digit format)

DATAFRAME: Ground Level, ACS C-R Function

       fips state_abbr pollutant   model  season      elevated        damage  \
0     01001         AL      pm25     AP2  annual  ground level  31153.706301   
1     01001         AL       so2     AP2  annual  ground level  17013.520087   
2     01001         AL       nox     AP2  annual  ground level   2253.330439   
3     01001         AL      pm25  EASIUR  annual  ground level  89706.549246   
4     01001         AL       so2  EASIUR  annual  ground level  17258.374396   
...     ...     

In [3]:
# ======================================================================================================================
# MARGINAL SOCIAL COSTS LOOKUP DICTIONARY: FOSSIL FUEL COMBUSTION
# ======================================================================================================================

# Ground Level, ACS C-R Function
lookup_health_rcm_ground_acs = create_lookup_nested(df_ground_acs)

# Ground Level, H6C C-R Function
lookup_health_rcm_ground_h6c = create_lookup_nested(df_ground_h6c)


print(f"""
======================================================================================================================
LOOKUP DICTIONARIES: 
======================================================================================================================

LOOKUP: Ground Level, ACS C-R Function
      
{lookup_health_rcm_ground_acs}
      
LOOKUP: Ground Level, H6C C-R Function

{lookup_health_rcm_ground_h6c}

""")


LOOKUP DICTIONARIES: 

LOOKUP: Ground Level, ACS C-R Function
      
{('01001', 'AL'): {'AP2': {'pm25': 47086.29274497018, 'so2': 25714.551575500598, 'nox': 3405.725652358905}, 'EASIUR': {'pm25': 135584.1516287581, 'so2': 26084.62894448302, 'nox': 6329.701868138744}, 'InMAP': {'pm25': 67883.09732096744, 'so2': 24580.745417471673, 'nox': 10870.179932201838}}, ('01003', 'AL'): {'AP2': {'pm25': 109765.43204731715, 'so2': 48039.31812529004, 'nox': 3401.1227873393104}, 'EASIUR': {'pm25': 134012.46834273523, 'so2': 24393.992099594616, 'nox': 5017.816687425366}, 'InMAP': {'pm25': 96474.90516452728, 'so2': 25899.049146301397, 'nox': 14802.485280196033}}, ('01005', 'AL'): {'AP2': {'pm25': 38649.93328780795, 'so2': 25592.050509626628, 'nox': 3713.844147185567}, 'EASIUR': {'pm25': 125825.5229662881, 'so2': 24471.261628419663, 'nox': 5614.993354462633}, 'InMAP': {'pm25': 40561.41686172057, 'so2': 23092.59358091167, 'nox': 9292.591830942962}}, ('01007', 'AL'): {'AP2': {'pm25': 43962.61236571297, '

## Emissions from Electricity Generation are considered "Elevated" or "High-Stack"

In [4]:
# ======================================================================================================================
# ELECTRICITY GENERATION:
# Load the data for Electricity Marginal Social Costs 
# Emissions from Electricity Generation are considered "Elevated" or "High-Stack"
# ======================================================================================================================

# Fossil Fuels
# Ground Level, ACS C-R Function
df_elevated_acs = process_rcm_data("rcm_msc_county_vsl857_usd2006_elevated_acs.csv", PROJECT_ROOT)

# Ground Level, H6C C-R Function
df_elevated_h6c = process_rcm_data("rcm_msc_county_vsl857_usd2006_elevated_h6c.csv", PROJECT_ROOT)

print(f"""
======================================================================================================================
Load the data for Electricity Marginal Social Costs
Emissions from Electricity Generation are considered "Elevated" or "High-Stack"
======================================================================================================================
Model: RCM
Output Area: FIPS
Geography: Counties
Pollutant: NOX,SO2,PM25
VSL: 8.57M (Inflated from 11.3M USD2021 to 8.57M USD2006)
Stack Height: high stack

C-R Function: ACS or H6C
======================================================================================================================
- Clean the data so that only annual values are retained. 
- Convert 'fips' to string and pad with leading zeros (to ensure a 5-digit format)

DATAFRAME: Elevated (High Stack), ACS C-R Function

{df_elevated_acs}
      
DATAFRAME: Elevated (High Stack), H6C C-R Function

{df_elevated_h6c}
      
""")


Load the data for Electricity Marginal Social Costs
Emissions from Electricity Generation are considered "Elevated" or "High-Stack"
Model: RCM
Output Area: FIPS
Geography: Counties
Pollutant: NOX,SO2,PM25
VSL: 8.57M (Inflated from 11.3M USD2021 to 8.57M USD2006)
Stack Height: high stack

C-R Function: ACS or H6C
- Clean the data so that only annual values are retained. 
- Convert 'fips' to string and pad with leading zeros (to ensure a 5-digit format)

DATAFRAME: Elevated (High Stack), ACS C-R Function

       fips state_abbr pollutant   model  season    elevated        damage  \
0     01001         AL      pm25     AP2  annual  high stack  17467.703269   
1     01001         AL       so2     AP2  annual  high stack  13603.292250   
2     01001         AL       nox     AP2  annual  high stack   2130.704503   
3     01001         AL      pm25  EASIUR  annual  high stack  47246.623498   
4     01001         AL       so2  EASIUR  annual  high stack  16113.086572   
...     ...        ...

In [5]:
# ======================================================================================================================
# MARGINAL SOCIAL COSTS LOOKUP DICTIONARY: ELECTRICITY GENERATION
# ======================================================================================================================

# Ground Level, ACS C-R Function
lookup_health_rcm_elevated_acs = create_lookup_nested(df_elevated_acs)

# Ground Level, H6C C-R Function
lookup_health_rcm_elevated_h6c = create_lookup_nested(df_elevated_h6c)


print(f"""
======================================================================================================================
LOOKUP DICTIONARIES: 
======================================================================================================================

LOOKUP: Elevated (High Stack), ACS C-R Function
      
{lookup_health_rcm_elevated_acs}
      
LOOKUP: Elevated (High Stack), H6C C-R Function

{lookup_health_rcm_elevated_h6c}

""")


LOOKUP DICTIONARIES: 

LOOKUP: Elevated (High Stack), ACS C-R Function
      
{('01001', 'AL'): {'AP2': {'pm25': 26401.01250686728, 'so2': 20560.269618588107, 'nox': 3220.386524918844}, 'EASIUR': {'pm25': 71409.42794223943, 'so2': 24353.619566758007, 'nox': 3803.129698171811}, 'InMAP': {'pm25': 34601.29484085969, 'so2': 22969.708659584834, 'nox': 7725.117026189864}}, ('01003', 'AL'): {'AP2': {'pm25': 33469.12072668417, 'so2': 20614.966755903017, 'nox': 2944.715566465875}, 'EASIUR': {'pm25': 68997.73180394845, 'so2': 23377.64682402977, 'nox': 3393.1626761766697}, 'InMAP': {'pm25': 41860.28800196884, 'so2': 24910.337358410674, 'nox': 8794.046669446569}}, ('01005', 'AL'): {'AP2': {'pm25': 23892.42687105433, 'so2': 21373.548919296, 'nox': 3092.4646295792154}, 'EASIUR': {'pm25': 67148.31356214755, 'so2': 24783.772515500626, 'nox': 3573.9814706233738}, 'InMAP': {'pm25': 31616.786532976515, 'so2': 21161.562116141118, 'nox': 7383.78082696081}}, ('01007', 'AL'): {'AP2': {'pm25': 24006.31269009